In [22]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime

In [23]:
#reading in the data

routes = pd.read_csv('GTFS data waikato/routes.txt')

trips = pd.read_csv('GTFS data waikato/trips.txt')

stops = pd.read_csv('GTFS data waikato/stop_times.txt', low_memory=False)

calendar = pd.read_csv("GTFS data waikato/calendar.txt")
exceptions = pd.read_csv('GTFS data waikato/calendar_dates.txt')


In [24]:
# reducing stops table to necessary data only

stops = stops[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'shape_dist_traveled','timepoint']]

# Select only the rows with the minimum and maximum 'stop_sequence' for each unique 'trip_id'
min_max_stops = stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmin()] #minimum
min_max_stops = pd.concat([min_max_stops, stops.loc[stops.groupby('trip_id')['stop_sequence'].idxmax()]]) #maximum

In [25]:
# Calculate the time difference between the first and last stop sequence for each trip_id and convert to hours
min_max_stops['trip_duration'] = min_max_stops.groupby('trip_id')['departure_time'].transform(lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).total_seconds() / 3600)

In [26]:
# calculate bus speed

min_max_stops['speed'] = min_max_stops['shape_dist_traveled']/(1000*min_max_stops['trip_duration'])

In [27]:
# Now select only entries for the last stop, remove entries for the 1st stop of the trip
min_max_stops = min_max_stops[min_max_stops['stop_sequence'] != 1]
min_max_stops.describe()

,trip_id,stop_id,stop_sequence,shape_dist_traveled,timepoint,trip_duration,speed
count,7067.000000,7.067000e+03,7067.000000,7067.000000,7067.000000,7067.000000,7067.000000
mean,3593.278336,1.400436e+07,37.516061,19987.086145,0.998868,0.705931,26.603834
std,2324.656770,2.778971e+03,15.613480,18288.929848,0.033629,0.407792,6.969872
min,1.000000,1.400106e+07,4.000000,3808.110805,0.000000,0.100000,13.987626
25%,1767.500000,1.400253e+07,28.000000,10406.747951,1.000000,0.416667,22.627042
50%,3534.000000,1.400253e+07,34.000000,12975.730830,1.000000,0.533333,25.352968
75%,5300.500000,1.400799e+07,42.000000,25882.780541,1.000000,0.833333,28.080060
max,21031.000000,1.400897e+07,77.000000,135294.119197,1.000000,2.566667,65.767724


In [28]:
# covert trip_id to integer which is currently as object and therefore is not possible to do a merge
min_max_stops['trip_id'] = min_max_stops['trip_id'].astype(int)

In [29]:
# Merge dataframes based on 'trip_id' column with 'outer' join
stops_trips = pd.merge(trips[['route_id', 'service_id', 'trip_id', 'trip_short_name', 'trip_headsign', 'direction_id',
                             'block_id', 'shape_id', 'operator_id']], min_max_stops, on=['trip_id'], how='outer')

In [30]:
# merge tirp, stop_times and routes data

routes_trips_stops = pd.merge(routes[['agency_id','route_id','route_short_name','route_long_name','route_type',
                                      'route_sort_order']], 
                              stops_trips[['route_id','service_id','trip_id','trip_short_name','trip_headsign', 'operator_id', 'shape_dist_traveled', 'trip_duration', 'speed']], on=['route_id'], how='outer')

In [31]:
# Manipulate calendar dataframe
calendar = pd.read_csv("GTFS data waikato/calendar.txt")
# Convert start_date and end_date columns to datetime
calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')

# Filter for year 2022
calendar = calendar[calendar['start_date'].dt.year == 2022]

# change the end date of service to last day of 2022 to avoid having data beyond 2022

# Define the threshold date
threshold_date = pd.to_datetime('2022-12-31')

# Replace values greater than the threshold with the threshold date
calendar.loc[calendar['end_date'] > threshold_date, 'end_date'] = threshold_date


In [32]:
new_calendar = pd.DataFrame()

new_calendar = pd.DataFrame()

for i in range(len(calendar)):
    start_dates = []

    dt_start = calendar.iloc[i,9]  #ith row of start_date
    dt_end = calendar.iloc[i,10]  #ith row of end date

    one_day = datetime.timedelta(1)
    start_dates = [dt_start]
    end_dates = []
    today = dt_start

    while today <= dt_end:
        tomorrow = today + one_day
        if tomorrow.month != today.month:
            start_dates.append(tomorrow)
            end_dates.append(today)
        today = tomorrow

    end_dates.append(dt_end)

    for j in range(len(start_dates)):
        new_row = calendar.iloc[i].copy()
        new_row['start_date'] = start_dates[j]
        new_row['end_date'] = end_dates[j]
        new_row['month'] = start_dates[j].month
        new_calendar = new_calendar.append(new_row,ignore_index=True)

new_calendar.to_csv("new_calendar.csv")
df4 = new_calendar
df4

/var/folders/j5/hf4njz5j1230dgtm2gwzfjrc0000gn/T/ipykernel_2054/3499032489.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
/var/folders/j5/hf4njz5j1230dgtm2gwzfjrc0000gn/T/ipykernel_2054/3499032489.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
/var/folders/j5/hf4njz5j1230dgtm2gwzfjrc0000gn/T/ipykernel_2054/3499032489.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_calendar = new_calendar.append(new_row,ignore_index=True)
/var/folders/j5/hf4njz5j1230dgtm2gwzfjrc0000gn/T/ipykernel_2054/3499032489.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pan

,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,month
0,WKO00_MTWThF_20220201_20220414,00_Secondary School Terms (1st Term only) (Wee...,1,1,1,1,1,0,0,2022-02-01,2022-02-28,2
1,WKO00_MTWThF_20220201_20220414,00_Secondary School Terms (1st Term only) (Wee...,1,1,1,1,1,0,0,2022-03-01,2022-03-31,3
2,WKO00_MTWThF_20220201_20220414,00_Secondary School Terms (1st Term only) (Wee...,1,1,1,1,1,0,0,2022-04-01,2022-04-14,4
3,WKO00_MTWThF_20221017_20221220,00_Secondary School Terms (2nd May) (Weekday),1,1,1,1,1,0,0,2022-10-17,2022-10-31,10
4,WKO00_MTWThF_20221017_20221220,00_Secondary School Terms (2nd May) (Weekday),1,1,1,1,1,0,0,2022-11-01,2022-11-30,11
...,...,...,...,...,...,...,...,...,...,...,...,...
402,WKO00_W_20220209_20220209,00_Mangakino_Taupo (Wednesday only),0,0,1,0,0,0,0,2022-02-09,2022-02-09,2
403,WKO00_W_20220323_20220323,00_Mangakino_Taupo (Wednesday only),0,0,1,0,0,0,0,2022-03-23,2022-03-23,3
404,WKO00_W_20220223_20220223,00_Mangakino_Taupo (Wednesday only),0,0,1,0,0,0,0,2022-02-23,2022-02-23,2
405,WKO00_W_20220126_20220126,00_Mangakino_Taupo (Wednesday only),0,0,1,0,0,0,0,2022-01-26,2022-01-26,1


In [33]:
new_calendar = pd.read_csv("GTFS data waikato/new_calendar.csv")
new_calendar[new_calendar['service_id'] == 'WKO00_MTWThF_20220201_20260306']
#??? the code does not work when the bus runs only for one day therefore 
#??? not all records have month and day count filled in

,Unnamed: 0,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,month
588,588,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-02-01,2022-02-28,2
589,589,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-03-01,2022-03-31,3
590,590,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-04-01,2022-04-30,4
591,591,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-05-01,2022-05-31,5
592,592,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-06-01,2022-06-30,6
593,593,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-07-01,2022-07-31,7
594,594,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-08-01,2022-08-31,8
595,595,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-09-01,2022-09-30,9
596,596,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-10-01,2022-10-31,10
597,597,WKO00_MTWThF_20220201_20260306,00_Year_Round_B_A_Christmas (Weekday),1,1,1,1,1,0,0,2022-11-01,2022-11-30,11


In [34]:
# Function to count number of days a bus runs betwwen start and end date considering the weekday columns

from datetime import timedelta

def count_days_excluding(row):
    current_date = row['start_date']
    day_count = 0
    
    while current_date <= row['end_date']:
        weekday_column = current_date.strftime('%A').lower() 
        if row[weekday_column] == 1:
            day_count += 1
        current_date += timedelta(days=1)
    
    return day_count

In [35]:
# Apply the function to each row and add the result as a new column
new_calendar['day_count'] = new_calendar.apply(count_days_excluding, axis=1)

AttributeError: 'str' object has no attribute 'strftime'

In [ ]:
new_calendar[new_calendar['service_id'] == 'WKO00_S_20221217_20221217']

In [ ]:
# Merge routes_trips_stors and new_calendar dataframes based on 'service_id' column with 'outer' join
trips_routes_calendar = pd.merge(new_calendar[['service_id','service_name','month','day_count']], routes_trips_stops, on=['service_id'], how='outer')

trips_routes_calendar.describe()


In [ ]:
trips_routes_calendar.head()